# Setup

In [2]:
install_load <- function (package1, ...)  {   

   # convert arguments to vector
   packages <- c(package1, ...)

   # start loop to determine if each package is installed
   for(package in packages){

       # if package is installed locally, load
       if(package %in% rownames(installed.packages()))
          do.call('library', list(package))

       # if package is not installed locally, download, then load
       else {
          install.packages(package)
          do.call("library", list(package))
       }
   } 
}

# Section 9.2

In [3]:
#install_load("tidyselect", "ggpubr", "parallel", "parallelMap", "randomForestSRC", "mlr", "tidyverse", "FSelector")
install_load("tidyselect", "parallel", "parallelMap", "randomForestSRC", "mlr", "tidyverse", "FSelector")


 randomForestSRC 2.9.3 
 
 Type rfsrc.news() to see new features, changes, and bug fixes. 
 

Loading required package: ParamHelpers
'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).

Attaching package: ‘mlr’

The following objects are masked from ‘package:randomForestSRC’:

    impute, subsample

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ purrr::partial() masks randomForestSRC::partial()
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘RWeka’



## Section 9.2.1

### Listing 9.1

In [4]:
data(Ozone, package = "mlbench")
ozoneTib <- as_tibble(Ozone)
names(ozoneTib) <- c("Month", "Date", "Day", "Ozone", "Press_height",
                     "Wind", "Humid", "Temp_Sand", "Temp_Monte",
                     "Inv_height", "Press_grad", "Inv_temp", "Visib")
head(ozoneTib)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


In [5]:
# missing data
map_dbl(ozoneTib, ~sum(is.na(.)))

Month         Date          Day        Ozone Press_height         Wind 
           0            0            0            5           12            0 
       Humid    Temp_Sand   Temp_Monte   Inv_height   Press_grad     Inv_temp 
          15            2          139           15            1           14 
       Visib 
           0

### Listing 9.2

In [6]:
ozoneClean <- mutate_all(ozoneTib, as.numeric) %>%
    filter(is.na(Ozone) == FALSE)

head(ozoneClean)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


### Listing 9.3

In [30]:
ozoneUntidy <- gather(ozoneClean, key = "Variable", value = "Value", -Ozone)

options(repr.plot.width=10, repr.plot.height=4)
ggplot(ozoneUntidy, aes(Value, Ozone)) +
    facet_wrap(~ Variable, scale = "free_x") + 
    geom_point() +
    geom_smooth() +
    geom_smooth(method = "lm", col = "red") + 
    theme_bw()

`geom_smooth()` using method = 'loess' and formula 'y ~ x'
Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing missing values (geom_point).”

## Section 9.2.2

### Listing 9.4

In [8]:
imputeMethod <- imputeLearner("regr.rpart")
ozoneImp <- impute(as.data.frame(ozoneClean), classes = list(numeric = imputeMethod))

### Listing 9.5

In [9]:
ozoneTask <- makeRegrTask(data = ozoneImp$data, target = "Ozone")
lin <- makeLearner("regr.lm")

### Listing 9.6

In [10]:
filterVals <- generateFilterValuesData(ozoneTask,
                                      method = "linear.correlation")
filterVals$data

name,type,filter,value
<chr>,<chr>,<fct>,<dbl>
Month,numeric,linear.correlation,0.05371420
Date,numeric,linear.correlation,0.08205094
Day,numeric,linear.correlation,0.04151444
Press_height,numeric,linear.correlation,0.58752354
Wind,numeric,linear.correlation,0.00468138
Humid,numeric,linear.correlation,0.45148094
Temp_Sand,numeric,linear.correlation,0.76977674
Temp_Monte,numeric,linear.correlation,0.74159034
Inv_height,numeric,linear.correlation,0.57563391


In [26]:
options(repr.plot.width=8, repr.plot.height=4)
plotFilterValues(filterVals) + theme_bw()

### Exercise 1

In [12]:
# use default method
filterValsRF <- generateFilterValuesData(ozoneTask, method = "randomForestSRC_importance")
filterValsRF$data

name,type,filter,value
<chr>,<chr>,<fct>,<dbl>
Month,numeric,randomForestSRC_importance,1.209377539
Date,numeric,randomForestSRC_importance,-0.068728895
Day,numeric,randomForestSRC_importance,0.034380750
Press_height,numeric,randomForestSRC_importance,0.834545977
Wind,numeric,randomForestSRC_importance,0.002808533
Humid,numeric,randomForestSRC_importance,2.112028890
Temp_Sand,numeric,randomForestSRC_importance,9.261543443
Temp_Monte,numeric,randomForestSRC_importance,7.261571976
Inv_height,numeric,randomForestSRC_importance,1.888358366


In [27]:
# Press_grad, Humid ranked higher in default filter 
plotFilterValues(filterValsRF) + theme_bw()

### Listing 9.7

In [28]:
# choose the best 6
ozoneFiltTaskBest6 <- filterFeatures(ozoneTask, 
                               fval = filterVals, abs = 6)
ozoneFiltFeatBest6 <- generateFilterValuesData(ozoneFiltTaskBest6, method = "linear.correlation")
p1 <- plotFilterValues(ozoneFiltFeatBest6) + theme_bw()

# choose the best 25%
ozoneFiltTaskTopQuartile <- filterFeatures(ozoneTask,
                               fval = filterVals, per = 0.25)
ozoneFiltFeatTopQuartile <- generateFilterValuesData(ozoneFiltTaskTopQuartile, method = "linear.correlation")
p2 <- plotFilterValues(ozoneFiltFeatTopQuartile) + theme_bw()


# choose above 20% correlation coefficient
ozoneFiltTaskTop20Percent <- filterFeatures(ozoneTask,
                               fval = filterVals, threshold = 0.2)
ozoneFiltFeatTop20Percent <- generateFilterValuesData(ozoneFiltTaskTop20Percent, method = "linear.correlation")
p3 <- plotFilterValues(ozoneFiltFeatTop20Percent) + theme_bw()

#figure <- ggarrange(p1, p2, p3,
#                    labels = c("A", "B", "C"),
#                    ncol = 3, nrow = 1)
#figure
p1
p2
p3

### Listing 9.8

In [15]:
filterWrapper <- makeFilterWrapper(learner = lin, 
                                  fw.method = "linear.correlation")

### Listing 9.9

In [16]:
lmParamSpace <- makeParamSet(
    makeIntegerParam("fw.abs", lower = 1, upper = 12))

gridSearch <- makeTuneControlGrid()

kFold <- makeResampleDesc("CV", iters = 10)

tunedFeats <- tuneParams(filterWrapper, task = ozoneTask,
                        resampling = kFold,
                        par.set = lmParamSpace, 
                        control = gridSearch)

tunedFeats

[Tune] Started tuning learner regr.lm.filtered for parameter set:
          Type len Def  Constr Req Tunable Trafo
fw.abs integer   -   - 1 to 12   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: fw.abs=1
[Tune-y] 1: mse.test.mean=25.6092916; time: 0.0 min
[Tune-x] 2: fw.abs=2
[Tune-y] 2: mse.test.mean=25.1296557; time: 0.0 min
[Tune-x] 3: fw.abs=3
[Tune-y] 3: mse.test.mean=24.9350078; time: 0.0 min
[Tune-x] 4: fw.abs=5
[Tune-y] 4: mse.test.mean=22.5324302; time: 0.0 min
[Tune-x] 5: fw.abs=6
[Tune-y] 5: mse.test.mean=20.8187261; time: 0.0 min
[Tune-x] 6: fw.abs=7
[Tune-y] 6: mse.test.mean=20.8468066; time: 0.0 min
[Tune-x] 7: fw.abs=8
[Tune-y] 7: mse.test.mean=21.0307872; time: 0.0 min
[Tune-x] 8: fw.abs=10
[Tune-y] 8: mse.test.mean=20.6572430; time: 0.0 min
[Tune-x] 9: fw.abs=11
[Tune-y] 9: mse.test.mean=20.5914330; time: 0.0 min
[Tune-x] 10: fw.abs=12
[Tune-y] 10: mse.test.mean=20.6665971; time: 0.0 min
[Tune] Result: fw.abs=11 : mse.test.mean=20

Tune result:
Op. pars: fw.abs=11
mse.test.mean=20.5914330

### Exercise 2

In [17]:
# repeat 9.8 with defaults
filterWrapperWithDefault <- makeFilterWrapper(learner = lin)

# repeat 9.9 with new wrapper
lmParamSpace2 <- makeParamSet(
    makeIntegerParam("fw.abs", lower = 1, upper = 12))

gridSearch2 <- makeTuneControlGrid()

kFold2 <- makeResampleDesc("CV", iters = 10)

tunedFeats2 <- tuneParams(filterWrapperWithDefault, task = ozoneTask,
                        resampling = kFold2,
                        par.set = lmParamSpace2, 
                        control = gridSearch2)

tunedFeats2

[Tune] Started tuning learner regr.lm.filtered for parameter set:
          Type len Def  Constr Req Tunable Trafo
fw.abs integer   -   - 1 to 12   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: fw.abs=1
[Tune-y] 1: mse.test.mean=26.6865859; time: 0.7 min
[Tune-x] 2: fw.abs=2
[Tune-y] 2: mse.test.mean=25.2339399; time: 0.6 min
[Tune-x] 3: fw.abs=3
[Tune-y] 3: mse.test.mean=24.9261765; time: 0.7 min
[Tune-x] 4: fw.abs=5
[Tune-y] 4: mse.test.mean=23.1192455; time: 0.7 min
[Tune-x] 5: fw.abs=6
[Tune-y] 5: mse.test.mean=21.5619711; time: 0.6 min
[Tune-x] 6: fw.abs=7
[Tune-y] 6: mse.test.mean=20.8134907; time: 0.7 min
[Tune-x] 7: fw.abs=8
[Tune-y] 7: mse.test.mean=20.7448640; time: 0.7 min
[Tune-x] 8: fw.abs=10
[Tune-y] 8: mse.test.mean=20.7511964; time: 0.6 min
[Tune-x] 9: fw.abs=11
[Tune-y] 9: mse.test.mean=20.8912085; time: 0.6 min
[Tune-x] 10: fw.abs=12
[Tune-y] 10: mse.test.mean=20.9781868; time: 0.6 min
[Tune] Result: fw.abs=8 : mse.test.mean=20.

Tune result:
Op. pars: fw.abs=8
mse.test.mean=20.7448640

In [18]:
# Filter wrapper method linear.correlation was much faster (0 min vs 0.6 min for each parameter)
# The default method chooses fewer features (10 vs 11)

### Listing 9.10

In [19]:
filteredTask <- filterFeatures(ozoneTask, fval = filterVals,
                              abs = unlist(tunedFeats$x))
filteredModel <- train(lin, filteredTask)

### Listing 9.11

In [20]:
featSelControl <- makeFeatSelControlSequential(method = "sfbs")

selFeats <- selectFeatures(learner = lin, task = ozoneTask,
                          resampling = kFold, control = featSelControl)
selFeats

[FeatSel] Started selecting features for learner 'regr.lm'
With control class: FeatSelControlSequential
Imputation value: Inf
[FeatSel-x] 1: 111111111111 (12 bits)
[FeatSel-y] 1: mse.test.mean=21.1523776; time: 0.0 min
[FeatSel-x] 2: 011111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.7183790; time: 0.0 min
[FeatSel-x] 2: 101111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.0599373; time: 0.0 min
[FeatSel-x] 2: 110111111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.0674451; time: 0.0 min
[FeatSel-x] 2: 111011111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.2866958; time: 0.0 min
[FeatSel-x] 2: 111101111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.1029836; time: 0.0 min
[FeatSel-x] 2: 111110111111 (11 bits)
[FeatSel-y] 2: mse.test.mean=22.0248498; time: 0.0 min
[FeatSel-x] 2: 111111011111 (11 bits)
[FeatSel-y] 2: mse.test.mean=22.2189459; time: 0.0 min
[FeatSel-x] 2: 111111101111 (11 bits)
[FeatSel-y] 2: mse.test.mean=21.7417699; time: 0.0 min
[FeatSel-x] 2: 111111110111 (11 bits)

FeatSel result:
Features (5): Month, Humid, Temp_Sand, Temp_Monte, Inv_height
mse.test.mean=20.6337091

### Listing 9.12

In [21]:
ozoneSelFeat <- ozoneImp$data[, c("Ozone", selFeats$x)]

ozoneSelFeatTask <- makeRegrTask(data = ozoneSelFeat, target = "Ozone")

wrapperModel <- train(lin, ozoneSelFeatTask)

## Section 9.2.4

### Listing 9.13

In [22]:
imputeMethod <- imputeLearner("regr.rpart")

imputeWrapper <- makeImputeWrapper(lin, 
                                 classes = list(numeric = imputeMethod))

featSelWrapper <- makeFeatSelWrapper(learner = imputeWrapper,
                                    resampling = kFold,
                                    control = featSelControl)

### Listing 9.14

In [23]:
ozoneTaskWithNAs <- makeRegrTask(data = ozoneClean, target = "Ozone")

kFold3 <- makeResampleDesc("CV", iters = 3)

parallelStartSocket(cpus = detectCores())

lmCV <- resample(featSelWrapper, ozoneTaskWithNAs, resampling = kFold3)

parallelStop()

lmCV

Warning message in makeTask(type = type, data = data, weights = weights, blocking = blocking, :
“Provided data is not a pure data.frame but from class tbl_df, hence it will be converted.”Starting parallelization in mode=socket with cpus=2.
Exporting objects to slaves for mode socket: .mlr.slave.options
Resampling: cross-validation
Measures:             mse       
Mapping in parallel: mode = socket; level = mlr.resample; cpus = 2; elements = 3.


Aggregated Result: mse.test.mean=22.7413325


Stopped parallelization. All cleaned up.


Resample Result
Task: ozoneClean
Learner: regr.lm.imputed.featsel
Aggr perf: mse.test.mean=22.7413325
Runtime: 1203.59

## Section 9.2.5

### Listing 9.15

In [24]:
wrapperModelData <- getLearnerModel(wrapperModel)

summary(wrapperModelData)


Call:
stats::lm(formula = f, data = d)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.7217  -2.8128  -0.2797   2.7788  13.8328 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.516e+01  1.950e+00  -7.775 8.24e-14 ***
Month       -2.895e-01  7.354e-02  -3.937 9.94e-05 ***
Humid        8.891e-02  1.310e-02   6.787 4.83e-11 ***
Temp_Sand    2.044e-01  4.252e-02   4.807 2.26e-06 ***
Temp_Monte   2.140e-01  5.555e-02   3.853 0.000139 ***
Inv_height  -5.521e-04  1.762e-04  -3.134 0.001871 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.482 on 355 degrees of freedom
Multiple R-squared:  0.6838,	Adjusted R-squared:  0.6794 
F-statistic: 153.6 on 5 and 355 DF,  p-value: < 2.2e-16


### Listing 9.16

In [31]:
options(repr.plot.width=10, repr.plot.height=6)
par(mfrow = c(2, 2))
plot(wrapperModelData)
par(mfrow = c(1, 1))

## Section 9.3

### Exercise 3

In [32]:
filterWrapperImputed <- makeFilterWrapper(learner = imputeWrapper,
                                         fw.method = "linear.correlation")

filterParam2 <- makeParamSet(
    makeIntegerParam("fw.abs",
                    lower = 1, upper = 12))

kFold4 <- makeResampleDesc("CV", iters = 3)

gridSearch3 <- makeTuneControlGrid()

tuneWrapper2 <- makeTuneWrapper(learner = filterWrapperImputed,
                               resampling <- kFold4,
                               par.set = filterParam2,
                               control = gridSearch3)

filterCV <- resample(tuneWrapper2, ozoneTask, resampling = kFold4)
filterCV

Resampling: cross-validation
Measures:             mse       
[Tune] Started tuning learner regr.lm.imputed.filtered for parameter set:
          Type len Def  Constr Req Tunable Trafo
fw.abs integer   -   - 1 to 12   -    TRUE     -
With control class: TuneControlGrid
Imputation value: Inf
[Tune-x] 1: fw.abs=1
[Tune-y] 1: mse.test.mean=30.8336884; time: 0.0 min
[Tune-x] 2: fw.abs=2
[Tune-y] 2: mse.test.mean=29.2405304; time: 0.1 min
[Tune-x] 3: fw.abs=3
[Tune-y] 3: mse.test.mean=26.9909012; time: 0.0 min
[Tune-x] 4: fw.abs=5
[Tune-y] 4: mse.test.mean=24.6061955; time: 0.0 min
[Tune-x] 5: fw.abs=6
[Tune-y] 5: mse.test.mean=20.6198385; time: 0.0 min
[Tune-x] 6: fw.abs=7
[Tune-y] 6: mse.test.mean=20.4270901; time: 0.0 min
[Tune-x] 7: fw.abs=8
[Tune-y] 7: mse.test.mean=20.6977387; time: 0.0 min
[Tune-x] 8: fw.abs=10
[Tune-y] 8: mse.test.mean=20.7930709; time: 0.0 min
[Tune-x] 9: fw.abs=11
[Tune-y] 9: mse.test.mean=20.7578274; time: 0.0 min
[Tune-x] 10: fw.abs=12
[Tune-y] 10: mse.test.mean

Resample Result
Task: ozoneImp$data
Learner: regr.lm.imputed.filtered.tuned
Aggr perf: mse.test.mean=21.6321467
Runtime: 48.0193

In [33]:
# much faster results compared to wrapper method (0.0 min per parameter, runtime 48 vs 1200!)
# mse = 21.63 vs 22.74 from wrapper method